# Tire kicker

Use this notebook to play with the code. The primary goal is to reproduce [Levi Wolf's Guinness delivery example](https://gist.github.com/ljwolf/e5927ab8c859ed477f496329c1ce19fc#file-guinness-py). 

### Handle imports

In [1]:
import geopandas as gpd
import pandas, numpy

In [2]:
import pandas

In [3]:
import sys
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt
import spopt

In [4]:
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [5]:
from spopt.route import engine, heuristic, utils

In [6]:
spopt.route.utils?

Type:        module
String form: <module 'spopt.route.utils' from '/home/dylan/projects/gsoc2025/spopt/spopt/route/utils.py'>
File:        ~/projects/gsoc2025/spopt/spopt/route/utils.py
Docstring:   <no docstring>

### Reproduce the guinness example

In [7]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

In [8]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In [9]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [10]:
dublin_pubs.shape

(551, 8)

In [11]:
gdf = dublin_pubs

In [12]:
depot = gdf.iloc[0,:]
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [13]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)
print("adding clients")

m.add_clients(
    locations = clients.geometry, 
    delivery = clients.demand,
    pickup = clients.supply,
    time_windows=None,
    service_times=(numpy.log(clients.demand)**2).astype(int)
)
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)

initializing model
adding clients
adding trucks


In [14]:
m.depot_location

(-6.28688, 53.341972)

#### From heuristic.py

_setup_graph is what needs attention, and buld_route_table is the operative function that is causing issues within _setup_graph

In the next section, I will troubleshoot util.build_route_table. I'm leaving `if http` chunk alone because I just want to get the basic version of the function (euclidean distances) to build the distances and durations table for the `pyvrp` solver.

In [15]:
from sklearn import metrics

In [16]:
import shapely

build_route_table accepts arguments for demand_sites and candidate_depots, then processes them like this:

These are constructed through the _setup_graph function, which only needs the all_lonlats object. It passes all_lonlats to build_route_table for both the demand_sites and candidate_depots argument.

all_lonlats is within solve(). Here, I've swapped out `self` for `m`. This block is found in `heuristic.py` at line 351.


In [17]:
all_lonlats = numpy.row_stack(
        (m.depot_location, shapely.get_coordinates(m.clients_.geometry))
    )

With all_lonlats defined, I can use it for demand_sits and candidate_depots. This chunk is found in `engine.py` at line 21.

In [18]:
all_lonlats

array([[-6.28688   , 53.341972  ],
       [-6.2605878 , 53.3305512 ],
       [-6.2288162 , 53.365756  ],
       ...,
       [-6.31899498, 53.36520717],
       [-6.25288447, 53.33808433],
       [-6.25872036, 53.34198563]])

Okay, so it looks like we're invoking a way to extract the coordinates from the geometry object twice. 

In [19]:
demand_sites = all_lonlats
candidate_depots = all_lonlats

ah yes, silence. Does this work now?

In [20]:
distances = metrics.pairwise_distances(
                    numpy.fliplr(numpy.deg2rad(demand_sites)), 
                    numpy.fliplr(numpy.deg2rad(candidate_depots)),
                    metric="haversine"
                ) * 6371000
durations = numpy.ceil((distances / 10) ** .75)

ah yes, silence

So if we back up a step, we still have our m object defined, can we use it to _setup_graph using our all_lonlats as inputs?

In [21]:
m._setup_graph(all_lonlats=all_lonlats)

/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:31: UserWarning: Failed to connect to routing engine... using haversine distance and (d/500)**.75 for durations
  warnings.warn(


In [22]:
import pyvrp
import geopandas

In [23]:
m.solve(stop=pyvrp.stop.MaxRuntime(60))

/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:31: UserWarning: Failed to connect to routing engine... using haversine distance and (d/500)**.75 for durations
  warnings.warn(


PyVRP v0.11.2

Solving an instance with:
    1 depot
    550 clients
    70 vehicles (6 vehicle types)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best

Search terminated in 60.37s after 159 iterations.
Best-found solution has cost 37985911.

Solution results
    # routes: 13
     # trips: 13
   # clients: 550
   objective: 37985911
    distance: 205951
    duration: 1117
# iterations: 159
    run-time: 60.37 seconds



/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and may not follow the road network.
  warnings.warn(
/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:72: UserWarning: Failed to connect to routing engine... constructed routes will be straight lines and 

In [24]:
m.explore?

Signature: m.explore()
Docstring: Make a webmap of the solution, colored by the route name. 
File:      ~/projects/gsoc2025/spopt/spopt/route/heuristic.py
Type:      method

In [25]:
m.write_result("./guinness")

TypeError: Object of type Timestamp is not JSON serializable

Something about these columns are not accepting my change from timestamp to str to bypass this JSON conversion error. 

In [ ]:
m.routes_, m.stops_ = utils.routes_and_stops(
            m.result_.best, m.model, m.clients_, m.depot_location, cost_unit=m.cost_unit
    )

In [ ]:
m.stops_

In [ ]:
m.explore()

In [ ]:
stops_for_map = m.stops_.copy()
stops_for_map["eta"] = stops_for_map.eta.astype(str)
stops_for_map["open_1"] = stops_for_map.open_1.astype(str)
stops_for_map["close_1"] = stops_for_map.close_1.astype(str)

In [ ]:
stops_for_map

In [ ]:
m.stops_.select_dtypes(include=["datetime64[ns]"])

In [ ]:
def explore(self):
    """
    Make a webmap of the solution, colored by the route name. 
    """
    if not hasattr(self, "routes_"):
        raise SpecificationError("must have solved the model to show the result")
    m = self.routes_.sort_values("route_name").explore(
        "route_name", categorical=True, tiles="CartoDB positron"
    )
    stops_for_map = self.stops_.copy()
    cols = ["eta", "open_1", "close_1"]
    for col in cols:
        stops_for_map[col] = self.stops_[col].astype(str)
    stops_for_map.explore(
        "route_name",
        m=m,
        legend=False,
        style_kwds=dict(color="black", radius=3, weight=1.5),
    )
    geopandas.GeoDataFrame(
        geometry=geopandas.points_from_xy(
            x=[self.depot_location[0]], y=[self.depot_location[1]], crs="epsg:4326"
        )
    ).explore(m=m, color="black", marker_type="marker")
    return m

In [ ]:
explore(m)

In [ ]:
solution = m.result_.best
model = m.model
target_geoms = m.clients_
depot_location = m.depot_location
cost_unit=1e-4

MIDNIGHT = pandas.to_datetime(
    "2030-01-02 00:00:00", format='%Y-%m-%d %H:%M:%S'
)

import geopandas

In [ ]:
def routes_and_stops(
    solution, 
    model, 
    target_geoms, 
    depot_location,
    cost_unit=1e-4
    ):
    """
    Calculate route geometries and stop etas/waypoint numbers from an input
    solution.

    Arguments
    ---------
    solution    :   pyvrp.Solution
        routing solution from running the pyvrp solver that describes the
        best available routes to satisfy the constraints and specifications 
        recorded in the `model`
    model   :   pyvrp.Model
        the model reflecting the problem specification that has been solved
        and recored in `solution`
    target_geoms    :   geopandas.GeoSeries/geopandas.GeoDataFrame
        the real-world longitude and latitude that correspond to the clients 
        recorded in the model. This should *not* include the depot location
        which is provided as a separate argument, unless the depot is also 
        located at a client. 
    depot_location  :   tuple
        the longitude and latitude values that correspond to the location
        of the delivery depot
    
    Returns
    -------
    two dataframes containing the routes and stops. the routes dataframe 
    will have one row per route, while the stops dataframe will be the same length 
    as the target_geoms input
    """
    assert solution.is_feasible(), "solution is infeasible, routes cannot be used."
    assert solution.is_complete(), "solution does not visit all required clients, routes cannot be used."
    n_routes = solution.num_routes()
    route_names = list(randomname.sample("adj/", "names/surnames/french",
        n=n_routes
    ))

    problem_data = model.data()

    # problem assumes all trucks have the same departure time
    # problem assumes that this is in minutes since 00:00:00
        
    route_lut = dict(zip(route_names, solution.routes()))
    stops = [
        (route_name, r.visits()) 
        for route_name, r in route_lut.items()
    ]

    stops = pandas.DataFrame(
        stops
    ).rename(
        columns={0:"route_name", 1:"stop_idx"}
    ).set_index("route_name")
    
    # calculate visit time, 
    # distances and durations are assumed constant over 
    # vehicle type
    duration_matrix, = problem_data.duration_matrices()
    distance_matrix, = problem_data.distance_matrices()
    # TODO: would it be helpful to have the running capacity? 
    def timedelta_from_visits(
        route, 
        duration_matrix=duration_matrix, 
        locations=model.locations
    ):  
        """
        This is a private function to estimate the time changes
        that evolve over a route using the model specific information,
        rather than using the osrm-provided durations on demand. 
        This is to account for any waiting that occurs at the stops. 
        """
        full_visits = [0, *route.visits(), 0]
        arrival_minutes = [route.start_time()]
        for stop_number, stop_idx in enumerate(full_visits[:-1]):
            next_stop_idx = full_visits[stop_number + 1]
            travel_duration = duration_matrix[stop_idx, next_stop_idx]
            # if service duration is not recorded, we assume
            # there is no service time (like, for a waypoint)
            service_duration = getattr(
                locations[stop_idx], "service_duration", 0
            )
            # once you're at stop_idx, you spend service_duration
            # there, and then spend travel_duration to get to the
            # next spot. So, the deltas should be 
            # [0, service_duration[1] + travel_duration[0,1], ...]
            # since the depot has service duration 0
            arrival_time = arrival_minutes[stop_number] + service_duration + travel_duration
            # if you arrive at the target before it's open, then you have to wait
            arrival_time = numpy.maximum(
                getattr(
                    locations[stop_idx],
                    "tw_early",
                    -numpy.inf
                ), arrival_time
            )
            arrival_minutes.append(arrival_time)
        tds = pandas.to_timedelta(arrival_minutes, unit='minutes')
        return tds
    
    stops['eta'] = pandas.Series(
        {name:timedelta_from_visits(r)[1:-1] + MIDNIGHT
        for name,r in route_lut.items()}
    )
    stops['stop_number'] = stops.stop_idx.apply(lambda x: numpy.arange(len(x))+1)
    
    big_stops = stops.explode(
        ["stop_idx", "stop_number", "eta"]
    )
    big_stops['target_uid'] = [
        model.locations[s].name for s in big_stops.stop_idx
    ]
    big_stops['stop_number'] = big_stops.groupby("route_name").cumcount().astype(int) + 1

    stop_output = target_geoms.copy(deep=True)
    stop_output = big_stops.reset_index().merge(
        target_geoms, left_on='target_uid', right_index=True,
        how='right'
        )
    stop_output['route_name'] = stop_output.route_name.fillna("unassigned")
    stop_output['stop_number'] = stop_output.stop_number.fillna(-1)
    stop_output = stop_output.sort_values(["route_name","stop_number"])
    stop_output = geopandas.GeoDataFrame(
        stop_output, 
        geometry='geometry', 
        crs=target_geoms.crs
    )

    route_data = []

    for name, group in stop_output.groupby("route_name"):
        route_obj = route_lut[name]
        group = group.sort_values("stop_number")
        coordinates = shapely.get_coordinates(group.geometry)
        shape, durations = engine.build_specific_route(
            numpy.vstack(
                (
                depot_location,
                coordinates, 
                depot_location
                )
            )
        )
        route_truck_type = route_obj.vehicle_type()
        truck_obj = model.vehicle_types[route_truck_type]
        deptime, rettime = pandas.to_timedelta([
                route_obj.start_time(),
                route_obj.end_time()
                ], unit="minutes"
                ) + MIDNIGHT

        route_data.append((
            name,
            truck_obj.name,
            route_obj.duration(),
            route_obj.distance(),
            route_obj.distance_cost() * cost_unit,
            route_obj.duration_cost() * cost_unit,
            truck_obj.fixed_cost * cost_unit,
            ( route_obj.distance_cost() 
             + route_obj.duration_cost() 
             + truck_obj.fixed_cost
            ) * cost_unit,
            deptime,
            rettime,
            round(float(route_obj.duration()) / truck_obj.max_duration * 100, 2),
            round(float(route_obj.delivery()[0]) / truck_obj.capacity[0] * 100, 2),
            round(float(route_obj.distance()) / truck_obj.max_distance * 100, 2),
            shape
        ))
    
    route_output = geopandas.GeoDataFrame(
        pandas.DataFrame(
            route_data,
            columns = [
                'route_name',
                'truck_type',
                'duration_min',
                'distance_m',
                'fuel_cost_€',
                'labor_cost_€',
                'truck_cost_€',
                'total_cost_€',
                'departure',
                'arrival',
                'utilization_time',
                'utilization_load',
                'utilization_rangelimit',
                'geometry'
            ]
        ),
        geometry='geometry', 
        crs=target_geoms.crs
        )
  
    return route_output, stop_output

Issue is happening because route_obj is not properly formatted. route_obj is created by route_lut, which requires the creation of route_names

In [ ]:
route_names

In [ ]:
problem_data = model.data()

route_lut = dict(zip(route_names, solution.routes()))
stops = [
    (route_name, r.visits()) 
    for route_name, r in route_lut.items()
]

stops = pandas.DataFrame(
    stops
).rename(
    columns={0:"route_name", 1:"stop_idx"}
).set_index("route_name")

duration_matrix, = problem_data.duration_matrices()
distance_matrix, = problem_data.distance_matrices()

In [ ]:
def timedelta_from_visits(
    route, 
    duration_matrix=duration_matrix, 
    locations=model.locations
):  
    """
    This is a private function to estimate the time changes
    that evolve over a route using the model specific information,
    rather than using the osrm-provided durations on demand. 
    This is to account for any waiting that occurs at the stops. 
    """
    full_visits = [0, *route.visits(), 0]
    arrival_minutes = [route.start_time()]
    for stop_number, stop_idx in enumerate(full_visits[:-1]):
        next_stop_idx = full_visits[stop_number + 1]
        travel_duration = duration_matrix[stop_idx, next_stop_idx]
        # if service duration is not recorded, we assume
        # there is no service time (like, for a waypoint)
        service_duration = getattr(
            locations[stop_idx], "service_duration", 0
        )
        # once you're at stop_idx, you spend service_duration
        # there, and then spend travel_duration to get to the
        # next spot. So, the deltas should be 
        # [0, service_duration[1] + travel_duration[0,1], ...]
        # since the depot has service duration 0
        arrival_time = arrival_minutes[stop_number] + service_duration + travel_duration
        # if you arrive at the target before it's open, then you have to wait
        arrival_time = numpy.maximum(
            getattr(
                locations[stop_idx],
                "tw_early",
                -numpy.inf
            ), arrival_time
        )
        arrival_minutes.append(arrival_time)
    tds = pandas.to_timedelta(arrival_minutes, unit='minutes')
    return tds

stops['eta'] = pandas.Series(
    {name:timedelta_from_visits(r)[1:-1] + MIDNIGHT
    for name,r in route_lut.items()}
)
stops['stop_number'] = stops.stop_idx.apply(lambda x: numpy.arange(len(x))+1)

big_stops = stops.explode(
    ["stop_idx", "stop_number", "eta"]
)
big_stops['target_uid'] = [
    model.locations[s].name for s in big_stops.stop_idx
]
big_stops['stop_number'] = big_stops.groupby("route_name").cumcount().astype(int) + 1

stop_output = target_geoms.copy(deep=True)
stop_output = big_stops.reset_index().merge(
    target_geoms, left_on='target_uid', right_index=True,
    how='right'
    )
stop_output['route_name'] = stop_output.route_name.fillna("unassigned")
stop_output['stop_number'] = stop_output.stop_number.fillna(-1)
stop_output = stop_output.sort_values(["route_name","stop_number"])
stop_output = geopandas.GeoDataFrame(
    stop_output, 
    geometry='geometry', 
    crs=target_geoms.crs
)

In [ ]:
route_lut

In [ ]:
for name, group in stop_output.groupby("route_name"):
    route_obj = route_lut[name]

In [ ]:
round(float(route_obj.delivery()[0]))

In [ ]:
route_truck_type = route_obj.vehicle_type()
truck_obj = model.vehicle_types[route_truck_type]

In [ ]:
routes_and_stops(solution, 
    model, 
    target_geoms, 
    depot_location,
    cost_unit)

In [ ]:
truck_obj

In [ ]:
route_obj

In [ ]:
print("Duration:", route_obj.duration(), type(route_obj.duration()))
print("Delivery:", route_obj.delivery(), type(route_obj.delivery()))
print("Distance:", route_obj.distance(), type(route_obj.distance()))

In [ ]:
print("duration:", truck_obj.max_duration, type(truck_obj.max_duration))
print("capacity:", truck_obj.capacity, type(truck_obj.capacity))
print("distance:", truck_obj.max_distance, type(truck_obj.max_distance))

In [ ]:
def build_route_table(demand_sites, candidate_depots, cost='distance', http=not has_bindings, database_path=_OSRM_DATABASE_FILE, port=5000):
    """
    Build a route table using OSRM, either over http or over py-osrm bindings
    """
    if isinstance(demand_sites, (geopandas.GeoSeries, geopandas.GeoDataFrame)):
        demand_sites = demand_sites.geometry.get_coordinates().values
    if isinstance(candidate_depots, (geopandas.GeoSeries, geopandas.GeoDataFrame)):
        candidate_depots = candidate_depots.geometry.get_coordinates().values
    if cost not in ("distance", "duration", "both"):
        raise ValueError(f"cost option '{cost}' not one of the supported options, ('distance', 'duration', 'both')")
    if http:
        try: 
            distances, durations = _build_route_table_http(demand_sites, candidate_depots, cost=cost, port=port)
        except (requests.ConnectionError, requests.JSONDecodeError):
            warnings.warn(
                "Failed to connect to routing engine... using haversine distance"
                " and (d/500)**.75 for durations"
            )
            distances = metrics.pairwise_distances(
                    numpy.fliplr(numpy.deg2rad(demand_sites)), 
                    numpy.fliplr(numpy.deg2rad(candidate_depots)),
                    metric="haversine"
                ) * 6371000
            durations = anumpy.ceil((distances / 10) ** .75)
    else:
        distances, durations = _build_route_table_pyosrm(
            demand_sites, candidate_depots, database_path=database_path 
        )
    for D in (distances, durations):
        if D is None:
            continue
        n_row, n_col = D.shape
        assert n_row == len(candidate_depots)
        assert n_col == len(demand_sites)
        no_route_available = numpy.isnan(D)
        D[no_route_available] = D[~no_route_available].sum()
    if cost == 'distance':
        return distances
    elif cost == 'duration':
        return durations
    elif cost == 'both':
        return distances, durations

In [ ]:
def _setup_graph(self, all_lonlats):
        """
        This sets up the graph pertaining to an inputted set of longitude and latitude coordinates. 

        Note that this assumes that there is a single vehicle profile.

        TODO: For multiple vehicle profiles, we would need to identify 
        the restricted and the base profiles, then update the model
        with an edge for each profile. 
        """
        raw_distances, raw_durations = engine.build_route_table(
            all_lonlats, all_lonlats, cost="both"
        )
        # how many minutes does it take to get from place to place?
        durations_by_block = numpy.ceil(raw_durations / 60)
        ##### WARNING!!!!!!! THIS IS A BUG IN OSRM #5855
        durations = numpy.clip(durations_by_block, 0, durations_by_block.max())
        distances = numpy.clip(raw_distances, 0, raw_distances.max()).round(0)

        duration_df = pandas.DataFrame(
            durations,
            index=[self.depot_name] + self.clients_.index.tolist(),
            columns=[self.depot_name] + self.clients_.index.tolist(),
        )
        distance_df = pandas.DataFrame(
            distances,
            index=[self.depot_name] + self.clients_.index.tolist(),
            columns=[self.depot_name] + self.clients_.index.tolist(),
        )
        for source_ix, source in enumerate(self.model.locations):
            for sink_ix, sink in enumerate(self.model.locations):
                self.model.add_edge(
                    source,
                    sink,
                    distance=distance_df.loc[source.name, sink.name].item(),
                    duration=duration_df.loc[
                        source.name, sink.name
                    ].item(),  # TODO: nogo zones
                )


In [ ]:
 def solve(self, stop=pyvrp.stop.NoImprovement(1e6), *args, **kwargs):
        """
        Solve a LastMile() instance according to the existing specification. 

        Parameters
        ----------
        stop    :   pyvrp.stop.StoppingCriterion
            A stopping rule that governs when the simulation will be ended. 
            Set to terminate solving after one million iterations with no improvement.


        Returns
        -------
        This LastMile() object, having added the results object to self.result_, as well
        as the routes and stops found to routes_ and stops_, respectively

        Notes
        -----
        other arguments and keyword arguments are passed directly to the pyvrp.Model.solve() method
        """
        if (not hasattr(self, "clients_")) | (not hasattr(self, "trucks_")):
            raise SpecificationError(
                "must assign both clients and trucks to" " solve a problem instance."
            )
        all_lonlats = numpy.row_stack(
            (self.depot_location, shapely.get_coordinates(self.clients_.geometry))
        )
        self._setup_graph(all_lonlats=all_lonlats)
        self.result_ = self.model.solve(stop=stop, *args, **kwargs)
        self.routes_, self.stops_ = utils.routes_and_stops(
            self.result_.best, self.model, self.clients_, self.depot_location, cost_unit=self.cost_unit
        )
        return self


#### From engine.py